In [5]:
# importamos las librerías necesarias
from imageai import Detection # librería de deteccion de imagenes
import sim          # librería para conectar con CoppeliaSim
import sympy as sp  # librería para cálculo simbólico
import numpy as np
from sympy.physics.mechanics import dynamicsymbols
import time
import math
import statistics

import cv2                      # opencv
import matplotlib.pyplot as plt # pyplot

from sympy import *

from sympy.physics.vector import init_vprinting
init_vprinting(use_latex='mathjax', pretty_print=False)

#######################################################################################
# CONEXIÓN
#######################################################################################

def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


# Función para abrir o cerrar la pinza remotamente:
def gripper(val):
# función que acciona el efector final remotamente
# val es Int con valor 0 ó 1 para desactivar o activar el actuador final. res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    res,retInts,retFloats,retStrings,retBuffer=sim.simxCallScriptFunction(clientID,
    "ROBOTIQ_85", sim.sim_scripttype_childscript,"gripper",[val],[],[],"", sim.simx_opmode_blocking)
    return res

# Cargar modelo de reconocimiento de imagen
yolo = Detection.ObjectDetection()
yolo.setModelTypeAsYOLOv3()
yolo.setModelPath("yolo.h5")
yolo.loadModel()

# Conexion con coppelia
clientID = connect(19999)

#######################################################################################
# VINCULACIÓN CON LOS OBJETOS DE COPPELIA
#######################################################################################

#ruedas
retCode,jointA=sim.simxGetObjectHandle(clientID,'OmniWheel45_typeA',sim.simx_opmode_blocking)
retCode,jointB=sim.simxGetObjectHandle(clientID,'OmniWheel45_typeB',sim.simx_opmode_blocking)

#dummy
retCode,dummy=sim.simxGetObjectHandle(clientID,'Dummy',sim.simx_opmode_blocking)

#objects
retCode,bottle=sim.simxGetObjectHandle(clientID,'Shape0',sim.simx_opmode_blocking)
retCode,bottle2=sim.simxGetObjectHandle(clientID,'Shape3',sim.simx_opmode_blocking)

#joints del brazo
ret,joint1=sim.simxGetObjectHandle(clientID,'joint1',sim.simx_opmode_blocking)
ret,joint2=sim.simxGetObjectHandle(clientID,'joint2',sim.simx_opmode_blocking)
ret,joint3=sim.simxGetObjectHandle(clientID,'joint3',sim.simx_opmode_blocking)
retCode,grip=sim.simxGetObjectHandle(clientID,'ROBOTIQ_85',sim.simx_opmode_blocking)

#base del brazo
retCode,base=sim.simxGetObjectHandle(clientID,'Cylinder',sim.simx_opmode_blocking)

#Camara
ret,camera=sim.simxGetObjectHandle(clientID,'camera',sim.simx_opmode_oneshot_wait)

#######################################################################################
# MOVIMIENTO RUEDAS
#######################################################################################
import time
#posicion de reposo del brazo
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 1.58, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -0.9, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_oneshot)
print(returnCode)

gripper(0) # Abrimos Pinza

# Busqueda de botella
going4Object = False

while not going4Object:

    errprCode,resolution,image = sim.simxGetVisionSensorImage(clientID,camera,0,sim.simx_opmode_streaming)
    time.sleep(0.2)
    errprCode,resolution,image = sim.simxGetVisionSensorImage(clientID,camera,0,sim.simx_opmode_buffer)
    
    # processing image
    frame = []
    frame = np.array(image,dtype = np.uint8)
    frame.resize([resolution[1],resolution[0],3])
    frame = cv2.flip(frame,0)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    

    ## predict yolo
    img, preds = yolo.detectObjectsFromImage(input_image=frame,
                      custom_objects=None, input_type="array",
                      output_type="array",
                      minimum_percentage_probability=35,
                      display_percentage_probability=False,
                      display_object_name=True)
    
    ## display predictions
    print(preds)
    if not going4Object:
       retCode = sim.simxSetJointTargetVelocity(clientID, jointA, -0.1,sim.simx_opmode_blocking)
       retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 0.1,sim.simx_opmode_blocking)
    for i in preds:
        if i["name"] == "bottle":
            if statistics.mean([i["box_points"][0],i["box_points"][2]])<750:
                retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 0.03,sim.simx_opmode_blocking)
                retCode = sim.simxSetJointTargetVelocity(clientID, jointB, -0.03,sim.simx_opmode_blocking)
                print("gira izquierda")
            elif statistics.mean([i["box_points"][0],i["box_points"][2]])>850:
                retCode = sim.simxSetJointTargetVelocity(clientID, jointA, -0.03,sim.simx_opmode_blocking)
                retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 0.03,sim.simx_opmode_blocking)
                print("gira derecha")
            else:
                going4Object = True
            time.sleep(2)
            retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 0,sim.simx_opmode_blocking)
            retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 0,sim.simx_opmode_blocking)

# Sacamos la posición para llegar a la botella desde el dummy de la pinza
returnCode,pos=sim.simxGetObjectPosition(clientID, bottle, -1, sim.simx_opmode_blocking)
x_b=pos[0]
y_b=pos[1]
z_b=pos[2]

print(pos)

returnCode,pos2=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
x_d=pos2[0]
y_d=pos2[1]
z_d=pos2[2]

while abs(x_d-x_b) > 0.35 or abs(y_d-y_b) > 0.35:
    
    retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 1,sim.simx_opmode_blocking)
    retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 1,sim.simx_opmode_blocking)
    
    time.sleep(1)
    retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 0,sim.simx_opmode_blocking)
    retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 0,sim.simx_opmode_blocking)
    
    returnCode,pos2=sim.simxGetObjectPosition(clientID, dummy, -1, sim.simx_opmode_blocking)
    x_d=pos2[0]
    y_d=pos2[1]
    z_d=pos2[2]
    
    print(pos2)

    
#hacia objeto
#returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 1.3, sim.simx_opmode_oneshot)
#print(returnCode)

#time.sleep(1)
#returnCode = sim.simxSetJointTargetPosition(clientID, joint2, 0.2, sim.simx_opmode_oneshot)
#print(returnCode)

#time.sleep(1)
#returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 1.31 , sim.simx_opmode_oneshot)
#print(returnCode)


#MOVIMIENTO HACIA EL OBJETO EN CUESTIÓN

# Sacamos la posición para llegar al cubo mirando donde está el dummyprox
returnCode,pos=sim.simxGetObjectPosition(clientID, bottle, -1, sim.simx_opmode_blocking)

returnCode,posB=sim.simxGetObjectPosition(clientID, base, -1, sim.simx_opmode_blocking)


#calcular angulo de la base
dy= (pos[1]-posB[1])#+0.03
dx= (pos[0]-posB[0])
angulo1=np.arctan2(-dy,dx)
print(angulo1)
xp=np.sqrt(pow(dx,2)+pow(-dy,2))
yp=posB[2]
b=xp
a=pos[2]-posB[2]

hip=np.sqrt(math.pow(a,2)+math.pow(b,2))
alfa=np.arctan2(a,b)
beta=np.arccos((math.pow(14,2)-math.pow(14,2)+math.pow(hip,2))/(2*14*hip))
angulo2=alfa+beta
print(angulo2)
angulo3=np.arccos((math.pow(14,2)+math.pow(14,2)-math.pow(hip,2))/(2*14*14))
print(angulo3)

returnCode = sim.simxSetJointTargetPosition(clientID, joint1, angulo1-0.295, sim.simx_opmode_oneshot)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, angulo2, sim.simx_opmode_oneshot)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, angulo3, sim.simx_opmode_oneshot)
#gripper(1)


# COGER OBJETO

gripper(0) # Abrimos Pinza
time.sleep(2)
gripper(1) # cerramos Pinza
time.sleep(2)

#Bloquear objeto en la pinza como hijo de la misma, así evitamos que se caiga
#https://www.coppeliarobotics.com/helpFiles/en/remoteApiFunctions.htm#simxSetObjectParent
[res, Robotiq]=sim.simxGetObjectHandle(clientID, 'ROBOTIQ_85_attachPoint', sim.simx_opmode_blocking);
[res, bottle]=sim.simxGetObjectHandle(clientID, 'Shape0', sim.simx_opmode_blocking);
returncode=sim.simxSetObjectParent(clientID, bottle,Robotiq, True, sim.simx_opmode_blocking);


#posicion de movimiento
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 1.58, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -0.9, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_oneshot)
print(returnCode)


#compartimento 2
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, -0.4, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -2.6, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 2.7 , sim.simx_opmode_oneshot)
print(returnCode)

# SOLTAR OBJETO

time.sleep(1)

#soltamos objeto, primero  borramos la propiedad de padre-hijo
returncode=sim.simxSetObjectParent(clientID, bottle,-1, False, sim.simx_opmode_blocking);
gripper(0)

# Posicion de reposo
time.sleep(2)
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 1.58, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -0.9, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 0, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)

retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 1,sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 1,sim.simx_opmode_blocking)
    
time.sleep(8)

retCode = sim.simxSetJointTargetVelocity(clientID, jointA, 0,sim.simx_opmode_blocking)
retCode = sim.simxSetJointTargetVelocity(clientID, jointB, 0,sim.simx_opmode_blocking)

conectado a 19999
1
1
1
[]
[]
[]
[]
[{'name': 'sports ball', 'percentage_probability': 82.92577266693115, 'box_points': [959, 433, 1045, 514]}]
[{'name': 'sports ball', 'percentage_probability': 94.57052946090698, 'box_points': [889, 429, 975, 512]}]
[{'name': 'sports ball', 'percentage_probability': 99.1845190525055, 'box_points': [807, 429, 890, 509]}]
[{'name': 'bottle', 'percentage_probability': 88.83863091468811, 'box_points': [1028, 368, 1079, 532]}, {'name': 'sports ball', 'percentage_probability': 94.83088254928589, 'box_points': [660, 424, 743, 505]}]
gira derecha
[{'name': 'bottle', 'percentage_probability': 70.481938123703, 'box_points': [821, 359, 879, 525]}, {'name': 'vase', 'percentage_probability': 49.05102849006653, 'box_points': [821, 359, 879, 525]}, {'name': 'sports ball', 'percentage_probability': 94.41685676574707, 'box_points': [461, 424, 542, 502]}]
[0.009879296645522118, -1.175302267074585, 0.03199995309114456]
[0.049137864261865616, -0.08364357054233551, 0.5203

In [125]:
#compartimento 1
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, 0.2, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -2, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 2.7 , sim.simx_opmode_oneshot)
print(returnCode)

0
0
0


In [141]:
#compartimento 2
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, -0.4, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -2.4, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 2.7 , sim.simx_opmode_oneshot)
print(returnCode)

0
0
0


In [37]:
#compartimento 3
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, -0.8, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -1.6, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 2.5 , sim.simx_opmode_oneshot)
print(returnCode)

0
0
0


In [29]:
#compartimento 4
returnCode = sim.simxSetJointTargetPosition(clientID, joint1, -1.5, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint2, -2, sim.simx_opmode_oneshot)
print(returnCode)

time.sleep(1)
returnCode = sim.simxSetJointTargetPosition(clientID, joint3, 2.7 , sim.simx_opmode_oneshot)
print(returnCode)

0
0
0
